In [ ]:
from sklearn.utils import shuffle
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
EPOCHS = 20 
BATCH_SIZE = 32
LEARNING_RATE = 0.02

train_loss_history = []
test_loss_history = []
acc_history = []

In [ ]:
print("Pobieranie danych z funkcji prepare_data()...")

# Zwraca ona już przeskalowane cechy "X" w zakresie [0, pi], więc z X nie musimy juz nic robic.
X_train, X_test, y_train_raw, y_test_raw = prepare_data()

# Funkcja prepare_data zwraca etykiety {0, 1}.
# Nasz model HybridModel używa pomiaru Z (wartości od -1 do 1).
# Konwertujemy: 0 -> -1 oraz 1 -> 1
y_train = 2 * y_train_raw - 1
y_test = 2 * y_test_raw - 1

# Upewniamy się, że typy danych to float32 (najlepsze dla PyTorch/NumPy)
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

print(f"Dane gotowe. Liczba próbek treningowych: {len(X_train)}")

In [ ]:
# PRZYGOTOWANIE OPTYMALIZATORA ADAM
# Tworzymy Tensor PyTorcha, który będzie przechowywał wagi i historię gradientów
weights_tensor = torch.tensor(weights, requires_grad=True, dtype=torch.float32)

# Używamy adam optimizer
optimizer = torch.optim.Adam([weights_tensor], lr=LEARNING_RATE)

print(f"Start treningu... Model: HybridModel, Epoki: {EPOCHS}, LR: {LEARNING_RATE}")

In [ ]:
# GŁÓWNA PĘTLA TRENINGOWA
for epoch in range(EPOCHS):
    # Tasowanie danych w każdej epoce
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train, random_state=epoch)
    
    epoch_loss = 0.0
    batches_count = 0
    
    # Iteracja po batchach
    for i in range(0, len(X_train), BATCH_SIZE):
        X_batch = X_train_shuffled[i:i + BATCH_SIZE]
        y_batch = y_train_shuffled[i:i + BATCH_SIZE]

        # Konwersja wag na NumPy dla Qiskita (bo Qiskit nie przyjmuje tensorów PyTorcha)
        current_weights_numpy = weights_tensor.detach().numpy()

        # forward
        # Predykcja modelu (zwraca wartości od -1 do 1)
        pred = qnn.forward(X_batch, current_weights_numpy)
        
        # backward
        # Obliczenie gradientów wag (zwraca numpy array)
        grads_numpy = qnn.backward(X_batch, current_weights_numpy)

        # MSE
        # Różnica: predykcja - prawda
        # Reshape tylko dla y_batch, bo pred wychodząc z QNN jest juz w kształcie (32,1)
        # Reshape od flatten rozni się tym ze flatten spłaszcza wszystko do jednego wymiaru i towrzy prostą listę liczb np. [1,2,3] 
        # reshape(-1,1) natomiast układa te same liczby w pionową kolumnę np. [[1], [2], [3]], reshape(3,1)- 3 wiersze i 1 kolumna 
        # reshape(-1,1) -1 oznacza ze bedzie 1 kolumna i parametr -1 zamienia sie automatycznie w BATCH_SIZE czyli 32
        diff = pred - y_batch.reshape(-1, 1)
        loss = np.mean(diff ** 2)
        
        # Chain Rule: pochodna MSE = 2 * (pred - y)
        # 2 * diff bo to pochodna z (pred - y)^2
        grad_modifier = 2 * diff
        
        # OBLICZANIE GRADIENTU
        # Usuwamy zbędny środkowy wymiar (z (32, 1, wagi) na (32, wagi))
        # Dzięki temu mamy prostą macierz: wiersz = próbka, kolumna = gradient wagi
        grads_2d = grads_numpy.reshape(grads_numpy.shape[0], -1)
        
        # Ważymy gradienty (Chain Rule)
        # Każdy wiersz gradientów mnożymy przez błąd danej próbki (grad_modifier)
        weighted_grads = grad_modifier * grads_2d
        
        # Obliczamy średnią po całym batchu (axis=0 to wiersze)
        # To daje nam jeden wektor gradientów do aktualizacji wag
        batch_grads_numpy = np.mean(weighted_grads, axis=0)
        
        # AKTUALIZACJA WAG
        # Czyścimy stare gradienty
        optimizer.zero_grad()
        
        # Dajemy nasz ręcznie policzony gradient do tensora PyTorcha
        weights_tensor.grad = torch.from_numpy(batch_grads_numpy)
        
        optimizer.step()
        
        epoch_loss += loss
        batches_count += 1

    # Zapisanie wytrenowanych wag z powrotem do zmiennej numpy
    weights = weights_tensor.detach().numpy()
    
    test_outputs = qnn.forward(X_test, weights)
    test_diff = test_outputs - y_test.reshape(-1, 1)
    test_loss = np.mean(test_diff ** 2)

    # zamiast predicted = (test_outputs > 0.5).astype(int).flatten()
    # bo nasze etykiety y_test to [-1, 1] a nie [0, 1] 
    # Używamy np.where: jeśli wynik > 0 to klasa 1, w przeciwnym razie klasa -1.
    predicted = np.where(test_outputs > 0, 1, -1).flatten()

    test_accuracy = np.mean(predicted == y_test.flatten())

    avg_loss = epoch_loss / batches_count
    train_loss_history.append(avg_loss)
    test_loss_history.append(test_loss)
    acc_history.append(test_accuracy)
    
    print(f"Epoch {epoch+1}/{EPOCHS} | Avg loss: {avg_loss:.4f} | Test Acc: {test_accuracy:.4f}")
    print("Trening zakończony.")

In [ ]:
test_outputs = qnn.forward(X_test, weights)

# Tutaj tez uzywamy np.where
predicted = np.where(test_outputs > 0, 1, -1).flatten()

print(confusion_matrix(y_test, predicted))
print(f1_score(y_test, predicted))

In [ ]:
epochs = range(1, EPOCHS + 1)

plt.figure(figsize=(14, 5))

# Plot 1: Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss_history, label='Train Loss', color='blue')
plt.plot(epochs, test_loss_history, label='Test Loss', color='red', linestyle='--')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot 2: Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, acc_history, label='Test Accuracy', color='green')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.show()